In [ ]:
!pip install --force-reinstall dist/alquimia_fair_forge-0.0.1.tar.gz -q

In [ ]:
from fair_forge.metrics import Conversational
from pydantic import SecretStr
from helpers.retriever import CustomRetriever

In [3]:
from getpass import getpass
judge_api_key = SecretStr(getpass("Please enter your Judge API key: "))

In [ ]:
metrics = Conversational.run(
    CustomRetriever,
    judge_api_key=judge_api_key
)

In [ ]:
def recreate_index(index_name: str, mapping: dict):
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
        print(f"Index '{index_name}' deleted.")
    es.indices.create(index=index_name, body=mapping)
    print(f"Index '{index_name}' created.")

In [ ]:
es = Elasticsearch(
    ELASTIC_URL,
    basic_auth=tuple(ELASTIC_AUTH),
)

In [ ]:
mapping_conversational = {
  "mappings": {
    "properties": {
      "session_id": {"type": "keyword"},
      "conversational_memory": {"type": "float"},
      "conversational_insight": {"type": "text"},
      "conversational_language": {"type": "float"},
      "conversational_quality_maxim": {"type": "float"},
      "conversational_quantity_maxim": {"type": "float"},
      "conversational_relation_maxim": {"type": "float"},
      "conversational_manner_maxim": {"type": "float"},
      "conversational_sensibleness": {"type": "float"},
      "conversational_thinkings": {"type": "text"},
      "qa_id": {"type": "keyword"},
      "assistant_id": {"type": "keyword"}
    }
  }
}
recreate_index(conversation_index, mapping_conversational)

In [ ]:
docs = []
for metric in metrics:
    docs.append({
            "_index": conversation_index,
            "_source": metric.model_dump()
    })

helpers.bulk(es, docs)
print(f"Indexed {len(docs)} documents.")